# Knockoff KNN Experiments

### Using breast cancer dataset
#### Dataset info
* Classes: 2 
* Features: 10

In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import load_breast_cancer
from sklearn.datasets import load_svmlight_file
from sklearn.metrics import accuracy_score

In [3]:
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
import keras.backend

Using TensorFlow backend.


In [4]:
def set_random_seed():
    random_seed = 0

    tf.set_random_seed(random_seed)
    np.random.seed(random_seed)

    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    sess = tf.Session(config=config)
    keras.backend.set_session(sess)
    keras.layers.core.K.set_learning_phase(0)
    #tensorflow.keras.backend.set_session(sess)
    sess.run(tf.global_variables_initializer())
    #auto_var.set_intermidiate_variable("sess", sess)
    random_state = np.random.RandomState(0)
    #auto_var.set_intermidiate_variable("random_state", random_state)

    return random_state

In [5]:
random_state = set_random_seed()

In [6]:
X, y = load_svmlight_file("./nnattack/datasets/files/breast-cancer")
X = X.todense() 
y[y==2] = 0
y[y==4] = 1
y = y.astype(int)

### Train test split
* Train sizes: 483
* Test sizes: 200
* Adversarial sizes: 100

In [7]:
idxs = np.arange(len(X))
random_state.shuffle(idxs)
x_train, x_test, y_train, y_test = X[idxs[:-200]], X[idxs[-200:]], y[idxs[:-200]], y[idxs[-200:]]

In [8]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

## Baseline Accuracy 

In [9]:
model = KNeighborsClassifier(n_neighbors=1)
model.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')

In [10]:
predictions = model.predict(x_test)
accuracy = accuracy_score(predictions,y_test)
print("Accuracy : %s" % "{0:.3%}".format(accuracy))

Accuracy : 95.000%


In [11]:
ori_x_test, ori_y_test=x_test, y_test
idxs = np.where(predictions == y_test)[0]
random_state.shuffle(idxs)
x_adv, y_adv = x_test[idxs[:100]], y_test[idxs[:100]]

## Knockoff Transferset Construction

In [20]:
from nnattack.attacks.nns import nn_attack
from nnattack.attacks.nns.nn_attack import NNAttack
from nnattack.attacks.nns.nn_attack import KNNRegionBasedAttackApprox
from nnattack.attacks.nns.direct import DirectAttack
from nnattack.attacks.nns.nn_attack import RevNNAttack

In [22]:
LINF_EPS = [0.01 * i for i in range(0, 81, 1)]
eps_list=LINF_EPS

### Constructing transferset with random sampling 483 datas

In [23]:
budget=483
dim=10

Random sampled 483 datas with total 683 datas

In [24]:
idxs = np.random.randint(len(X), size=budget)

In [25]:
x_train_rand_transfer = X[idxs]

In [26]:
scaler = MinMaxScaler()
x_train_rand_transfer = scaler.fit_transform(x_train_rand_transfer)

Using baseline model for getting labels

In [27]:
y_train_rand_transfer = model.predict(x_train_rand_transfer)

Training with transfer set

In [28]:
model_rand_transfer = KNeighborsClassifier(n_neighbors=1)
model_rand_transfer.fit(x_train_rand_transfer, y_train_rand_transfer)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')

In [29]:
predictions = model_rand_transfer.predict(x_test)
accuracy = accuracy_score(predictions,y_test)
print("Accuracy : %s" % "{0:.3%}".format(accuracy))

Accuracy : 93.500%


### Constructing transferset with 483 adversarial examples with RBA-APPROX Attack

In [30]:
x_adv_transfer = x_train
attack_model = KNNRegionBasedAttackApprox(
                    x_adv_transfer,
                    y_train,
                    n_searches=50,
                    n_neighbors=1,
                    ord=np.inf
                )

perturbs_rba_approx = attack_model.perturb(x_adv_transfer, y=y_train, eps=eps_list)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    9.9s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   42.9s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 483 out of 483 | elapsed:  1.8min finished


In [31]:
if attack_model is not None and hasattr(attack_model, 'perts'):
    perts_rba_approx = attack_model.perts
else:
    perts_rba_approx = np.zeros_like(x_adv_transfer)
    for pert in perturbs_rba_approx:
        pred = model.predict(x_adv_transfer + pert)
        for i in range(len(pred)):
            if (pred[i] != y_train[i]) and np.linalg.norm(perts_rba_approx[i])==0:
                perts_rba_approx[i] = pert[i]

In [32]:
perts_rba_approx = perts_rba_approx.astype(float)

Using baseline model to get label with data with noise

In [33]:
y_adv_transfer = model.predict(x_adv_transfer+perts_rba_approx*0.5)

In [44]:
model_adv_transfer = KNeighborsClassifier(n_neighbors=1)
model_adv_transfer.fit(x_adv_transfer+perts_rba_approx*0.5, y_adv_transfer)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')

In [45]:
predictions = model_adv_transfer.predict(x_test)
accuracy = accuracy_score(predictions,y_test)
print("Accuracy : %s" % "{0:.3%}".format(accuracy))

Accuracy : 96.000%


### Constructing the transfer set with adding gaussian noice to training data

In [46]:
noise = np.random.normal(0,1,(budget, dim))

In [47]:
x_train_noise = x_train + noise
y_train_noise = model.predict(x_train_noise)

model_with_noise = KNeighborsClassifier(n_neighbors=1)
model_with_noise.fit(x_train_noise, y_train_noise)

predictions = model_with_noise.predict(x_test)
accuracy = accuracy_score(predictions,y_test)
print("Accuracy : %s" % "{0:.3%}".format(accuracy))

Accuracy : 53.000%


### Constructing the transfer set with DirectAttack

In [48]:
x_adv_dir = x_train
direct_attack = DirectAttack(n_neighbors=1, ord=np.inf)
direct_attack.fit(x_adv_dir, y_train)
perturbs_direct_attack = direct_attack.perturb(x_adv_dir, y_train, eps=eps_list)
perts_direct_attack = np.zeros_like(x_adv_dir)
for pert in perturbs_direct_attack:
    pred = model.predict(x_adv_dir + pert)
    for i in range(len(pred)):
        if (pred[i] != y_train[i]) and np.linalg.norm(perts_direct_attack[i])==0:
            perts_direct_attack[i] = pert[i]
perts_direct_attack = perts_direct_attack.astype(float)

In [49]:
y_adv_transfer_dir = model.predict(x_adv_dir+perts_direct_attack*0.9)

In [50]:
model_adv_transfer_dir = KNeighborsClassifier(n_neighbors=1)
model_adv_transfer_dir.fit(x_adv_dir+perts_direct_attack*0.9, y_adv_transfer_dir)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')

In [51]:
predictions = model_adv_transfer_dir.predict(x_test)
accuracy = accuracy_score(predictions,y_test)
print("Accuracy : %s" % "{0:.3%}".format(accuracy))

Accuracy : 97.000%
